In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import time

In [16]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('test.csv')

# Dividir los datos
X = df.drop('Cancer Present', axis=1)
y = df['Cancer Present']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)


Entrenamiento del modelo secuencial

In [17]:

from sklearn.neighbors import KNeighborsClassifier

# Entrenar el modelo
modelo_knn = KNeighborsClassifier(n_neighbors=20)
modelo_knn.fit(X_train, y_train)

#tiempo de ejecucion para la prediccion

%time y_pred_secuencial = modelo_knn.predict(X_test)


CPU times: total: 52.6 s
Wall time: 1min 5s


Entrenamiento del modelo con procesamiento paralelo

In [18]:
import numpy as np
import joblib

In [19]:
import timeit

# Función wrapper para la predicción
def predict_batch(model, data_batch):
    return model.predict(data_batch)


def parallel_prediction():
    # Configurar el backend de joblib para usar hilos
    with joblib.parallel_backend('threading', n_jobs=-1):
        
        # Dividir el conjunto de prueba en lotes para la paralelización
        test_batches = np.array_split(X_test, joblib.cpu_count())
        
        # Realizar la predicción paralela
        y_pred_batches = joblib.Parallel()(joblib.delayed(predict_batch)(modelo_knn, batch) for batch in test_batches)
        
        # Concatenar los resultados de la predicción paralela
        y_pred_parallel = np.concatenate(y_pred_batches, axis=0)
    
    return y_pred_parallel


# Ejecutar la función y medir el wall time
start_time_parallel = timeit.default_timer()

prediction_result = parallel_prediction()

end_time_parallel = timeit.default_timer()

wall_time_parallel = end_time_parallel - start_time_parallel
print(f'Wall time de ejecución paralela: {wall_time_parallel} segundos')


Wall time de ejecución paralela: 23.853410999989137 segundos
